# Import libraries and variables

In [1]:
import json
import os
from sortedcontainers import SortedDict

%store -r output_activity_dir
%store -r activity_all_file
%store -r output_person_dir
%store -r persons_all_file

%store -r events_nonmoma
%store -r json_suffix

%store -r output_datavis_venue_exhibitions 

%store -r output_datavis_dir

%store -r datavis_network 
%store -r org_artist_links_nonmoma 
%store -r org_artist_links_10thstreet




# Create a forced network diagram

diagram with links between artists and organisations (via inclusion in exhibitions)

In [2]:

with open(os.path.join(output_activity_dir, activity_all_file), 'r') as json_file:
  
    nodes = []
    links = []

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")

        nodes_ids  = [x['id'] for x in nodes]
       
        if org == "The ":
            continue
        if org == "MoMA PS1":
            continue
        if org not in nodes_ids:
            nodes.append({"id": org, "group": 1})

          
        influenced_by = event.get("influenced_by")
        for person in influenced_by:
            if person not in nodes_ids:
                nodes.append({"id": person, "group": 2})

            #links_sources  = [x['source'] for x in links]

            links.append({"source": org, "target": person, "value": 1})

    with open(os.path.join(output_datavis_dir , datavis_network, org_artist_links_nonmoma), 'w') as file:
        file.write(json.dumps({"nodes": nodes, "links": links},indent=2))
        

In [14]:
# create dictionary of person id and name from persons_all

persons = {}
with open(os.path.join(output_person_dir , persons_all_file ), 'r') as json_file: 
    data = json.load(json_file)
    for person in data.get("persons"):
        id = person.get("id")
        name = person.get("_label")
        persons[id] = name

print(persons)
        


{'https://www.moma.org/data/person/57896': 'Else Gabriel', 'https://www.moma.org/data/person/3721': 'Ann McCoy', 'https://www.moma.org/data/person/29289': 'Tony Berardi', 'https://www.moma.org/data/person/56784': 'Jason Rosenberg', 'https://www.moma.org/data/person/23684': 'Clara Gaggero', 'https://www.moma.org/data/person/59471': 'Javier Velasco', 'https://www.moma.org/data/person/729': 'Raymond Breinin', 'https://www.moma.org/data/person/35750': 'Edward H. Mitchell', 'https://www.moma.org/data/person/28861': 'Loomis Dean', 'https://www.moma.org/data/person/32095': 'François Stahly', 'https://www.moma.org/data/person/6523': 'Betsey Johnson', 'https://www.moma.org/data/person/3371': 'Mon Levinson', 'https://www.moma.org/data/person/683': 'Samuel Bourne', 'https://www.moma.org/data/person/13662': 'Dona Nelson', 'https://www.moma.org/data/person/59021': 'Despina Stamos', 'https://www.moma.org/data/person/21713': 'Christian Jankowski', 'https://www.moma.org/data/person/379': 'Thomas Bayrl

In [16]:
with open(os.path.join(output_activity_dir, activity_all_file), 'r') as json_file:  
    nodes = []
    links = []

    

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")

        nodes_ids  = [x['id'] for x in nodes]
       # , "MoMA PS1"
        list_orgs = ['Tanager Gallery', "Brata Gallery", "Hansa Gallery"]
        if org not in list_orgs:
            continue
        
        if org not in nodes_ids:
            nodes.append({"id": org, "group": 1})

        influenced_by = event.get("influenced_by")
        for person in influenced_by:

            # get person_name 
            name = ""
            if person.lower() in persons:
                name = persons[person.lower()]
            
            if person not in nodes_ids:
                nodes.append({"id": person, "group": 2, "name": name})

            links.append({"source": org, "target": person, "value": 1})


    list_links_persons  = [x['target'] for x in links]
    list_nodes_ids = [x['id'] for x in nodes]

    nonmoma_links = []
    moma_links = []

    for link in links:
        if link.get("source") not in ["MoMA PS1", "The Museum of Modern Art"]:
            nonmoma_links.append(link)
        else:
            moma_links.append(link)

    list_persons_nonmoma = [x['target'] for x in nonmoma_links]
    list_persons_moma = [x['target'] for x in moma_links]

    # get persons in moma list that are not in nonmoma list and then remove links including them 
    # and remove corresponding nodes

    list_persons_uniquetomoma= []
    for person in list_persons_moma:
        if person not in list_persons_nonmoma:
            list_persons_uniquetomoma.append(person)

    
    i = 0
    while i < len(links):
        link = links[i]
        if "target" in link: 
            person = link.get("target")
            if person in list_persons_uniquetomoma:
                
                del links[i]
        i += 1

    i = 0
 

    with open(os.path.join(output_datavis_dir , datavis_network, org_artist_links_10thstreet), 'w') as file:
        file.write(json.dumps({"nodes": nodes, "links": links},indent=2))


 
        

# Data visualisation - 10th Street exhibition venues

The resulting data file has been loaded into a notebook containing a d3.js force-directed graph visualisation:

https://observablehq.com/d/5be8a88ffb1207ed